In [ ]:
!pip install pypdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.6/304.6 kB 7.9 MB/s eta 0:00:00


In [ ]:
!pip install langchain transformers accelerate sentence-transformers faiss-cpu gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
!pip install -U langchain langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.65
    Uninstalling langchain-core-0.3.65:
      Successfully uninstalled langchain-core-0.3.65
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25


In [ ]:
from langchain_community.document_loaders import PyPDFLoader


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline


In [ ]:
from langchain.document_loaders import PyPDFLoader
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
import gradio as gr
import tempfile


In [ ]:
def load_pdf(filepath):
    loader = PyPDFLoader(filepath)
    return loader.load_and_split()


In [ ]:
def get_embedder():
    return SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
class FAISSStore:
    def __init__(self, embedder):
        self.embedder = embedder
        self.index = None
        self.docs = []

    def build_index(self, docs):
        texts = [doc.page_content for doc in docs]
        self.docs = docs
        vectors = self.embedder.encode(texts)
        self.index = faiss.IndexFlatL2(vectors.shape[1])
        self.index.add(np.array(vectors))

    def query(self, question, k=5):
        q_vec = self.embedder.encode([question])
        D, I = self.index.search(np.array(q_vec), k)
        return [self.docs[i] for i in I[0]]


In [ ]:
def load_llm(model_name="google/flan-t5-large"):
    from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)
    return HuggingFacePipeline(pipeline=pipe)


In [ ]:
from langchain.vectorstores import FAISS as LangChainFAISS
from langchain.embeddings import HuggingFaceEmbeddings

def get_retrieved_context(store, question):
    texts = [doc.page_content for doc in store.docs]
    metadata = [{"source": f"chunk_{i}"} for i in range(len(store.docs))]

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    langchain_faiss = LangChainFAISS.from_texts(texts, embeddings, metadatas=metadata)
    retriever = langchain_faiss.as_retriever(search_kwargs={"k": 5})

    docs = retriever.get_relevant_documents(question)
    context = "\n\n".join([doc.page_content for doc in docs])
    return context


In [ ]:
embedder = get_embedder()
llm = load_llm()

def handle_query(file, question):
    try:
        import tempfile
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as temp:
            temp.write(open(file.name, "rb").read())
            pdf_path = temp.name

        docs = load_pdf(pdf_path)

        if len(docs) < 5 or sum(len(d.page_content.split()) for d in docs) < 200:
            return "⚠️ That doesn't look like a research paper. Try uploading something more technical?"

        store = FAISSStore(embedder)
        store.build_index(docs)

        # Get context
        context = get_retrieved_context(store, question)

        # Informal prompt
        prompt = f"""Hey! Here's a paper I'm reading. Based on this, could you casually answer the question:

Question: {question}

Paper bits:
{context}

Try to be helpful, clear, and chill. If the answer isn't really in there, just say that politely.
"""

        response = llm(prompt)
        return response or "Hmm, couldn't find a good answer. Try rephrasing?"

    except Exception as e:
        return f"🔥 Error: {str(e)}"


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-10-1614934315.py:6: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  return HuggingFacePipeline(pipeline=pipe)


In [ ]:
import gradio as gr

# ✅ Professional minimal CSS
professional_css = """
body {
    background-color: #f8f9fa;
    font-family: 'Segoe UI', sans-serif;
}

.gradio-container {
    max-width: 800px;
    margin: auto;
    padding: 20px;
    background-color: white;
    border-radius: 8px;
    box-shadow: 0 0 15px rgba(0, 0, 0, 0.08);
}

h1, h2 {
    text-align: center;
    color: #343a40;
}

textarea, input[type="file"], input[type="text"] {
    border: 1px solid #ced4da;
    border-radius: 4px;
    padding: 10px;
    font-size: 15px;
    background-color: #ffffff;
}

button {
    background-color: #0069d9;
    color: white;
    border: none;
    border-radius: 4px;
    padding: 10px 18px;
    font-size: 15px;
    cursor: pointer;
}

button:hover {
    background-color: #0056b3;
}
"""

# ✅ Build the app layout
with gr.Blocks(css=professional_css) as demo:
    gr.Markdown("## AI Research Co-Pilot")
    gr.Markdown("Upload a research paper and ask questions about it using RAG + Hugging Face.")

    with gr.Row():
        file_input = gr.File(label="Upload Research Paper (PDF)")
        question_input = gr.Textbox(label="Ask a Question", placeholder="e.g. What is the main contribution?")

    output = gr.Textbox(label="Answer", lines=6)
    submit_button = gr.Button("Generate Answer")

    def run_query(file, question):
        return handle_query(file, question)

    submit_button.click(fn=run_query, inputs=[file_input, question_input], outputs=output)

demo.launch()



It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://03516bcc4d5ea67cf0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
